In [1]:
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [5]:
X = np.array(sequences)

In [6]:
y = to_categorical(labels).astype(int)

In [8]:
loss = []
accuracies = []
precisions = []
recalls = []

kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X):
    
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    features_train, features_test = X[train_index], X[test_index]
    labels_train, labels_test = y[train_index], y[test_index]
    
    # Construct the required LSTM model.
    model = Sequential() # changed activation to tanh
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(LSTM(64, return_sequences=False, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    # Display the success message.
    print("Model Created Successfully!")
    
    # Create an Instance of Early Stopping Callback
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['categorical_accuracy',Precision(),Recall()]) 
    
    model_training_history = model.fit(features_train, labels_train, epochs=300, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     callbacks = [early_stopping_callback])

    # Evaluate the trained model.
    model_evaluation_history = model.evaluate(features_test, labels_test)
    
    eval_loss, eval_acc, eval_prec, eval_rec = model_evaluation_history
    loss.append(eval_loss)
    accuracies.append(eval_acc)
    precisions.append(eval_prec)
    recalls.append(eval_rec)

Model Created Successfully!
Epoch 1/300
112/112 [==============================] - 40s 186ms/step - loss: 2.6416 - categorical_accuracy: 0.1049 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.6044 - val_categorical_accuracy: 0.1875 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/300
112/112 [==============================] - 14s 123ms/step - loss: 2.6039 - categorical_accuracy: 0.1406 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.5676 - val_categorical_accuracy: 0.1339 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/300
112/112 [==============================] - 14s 120ms/step - loss: 2.5456 - categorical_accuracy: 0.1161 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 2.5030 - val_categorical_accuracy: 0.1071 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/300
112/112 [==============================] - 13s 121ms/step - loss: 2.4695 - categorical_accuracy: 0.1250 - precision_1: 0.0000e+00 - 

112/112 [==============================] - 18s 162ms/step - loss: 1.1305 - categorical_accuracy: 0.6964 - precision_1: 0.9792 - recall_1: 0.1049 - val_loss: 1.2887 - val_categorical_accuracy: 0.6071 - val_precision_1: 1.0000 - val_recall_1: 0.0982
Epoch 62/300
112/112 [==============================] - 16s 141ms/step - loss: 1.1094 - categorical_accuracy: 0.7277 - precision_1: 0.9811 - recall_1: 0.1161 - val_loss: 1.2007 - val_categorical_accuracy: 0.6518 - val_precision_1: 1.0000 - val_recall_1: 0.0982
Epoch 63/300
112/112 [==============================] - 13s 118ms/step - loss: 1.1149 - categorical_accuracy: 0.7299 - precision_1: 0.9825 - recall_1: 0.1250 - val_loss: 1.2817 - val_categorical_accuracy: 0.6161 - val_precision_1: 1.0000 - val_recall_1: 0.1071
Epoch 64/300
112/112 [==============================] - 16s 140ms/step - loss: 1.0841 - categorical_accuracy: 0.7321 - precision_1: 0.9861 - recall_1: 0.1585 - val_loss: 1.2244 - val_categorical_accuracy: 0.6429 - val_precision_1:

112/112 [==============================] - 16s 139ms/step - loss: 0.5821 - categorical_accuracy: 0.8929 - precision_1: 0.9719 - recall_1: 0.6942 - val_loss: 0.6910 - val_categorical_accuracy: 0.8214 - val_precision_1: 0.9324 - val_recall_1: 0.6161
Epoch 124/300
112/112 [==============================] - 15s 138ms/step - loss: 0.5061 - categorical_accuracy: 0.9420 - precision_1: 0.9941 - recall_1: 0.7567 - val_loss: 0.6297 - val_categorical_accuracy: 0.8571 - val_precision_1: 0.9398 - val_recall_1: 0.6964
Epoch 125/300
112/112 [==============================] - 16s 139ms/step - loss: 0.4864 - categorical_accuracy: 0.9375 - precision_1: 0.9915 - recall_1: 0.7812 - val_loss: 0.6164 - val_categorical_accuracy: 0.9196 - val_precision_1: 0.9630 - val_recall_1: 0.6964
Epoch 126/300
112/112 [==============================] - 15s 136ms/step - loss: 0.4916 - categorical_accuracy: 0.9308 - precision_1: 0.9858 - recall_1: 0.7723 - val_loss: 0.6647 - val_categorical_accuracy: 0.8571 - val_precision

112/112 [==============================] - 15s 137ms/step - loss: 0.2528 - categorical_accuracy: 0.9754 - precision_1: 0.9906 - recall_1: 0.9442 - val_loss: 0.4566 - val_categorical_accuracy: 0.8661 - val_precision_1: 0.9126 - val_recall_1: 0.8393
Epoch 186/300
112/112 [==============================] - 16s 145ms/step - loss: 0.2427 - categorical_accuracy: 0.9710 - precision_1: 0.9929 - recall_1: 0.9420 - val_loss: 0.3937 - val_categorical_accuracy: 0.9375 - val_precision_1: 0.9515 - val_recall_1: 0.8750
Epoch 187/300
112/112 [==============================] - 16s 147ms/step - loss: 0.2487 - categorical_accuracy: 0.9732 - precision_1: 0.9953 - recall_1: 0.9464 - val_loss: 0.3788 - val_categorical_accuracy: 0.9196 - val_precision_1: 0.9429 - val_recall_1: 0.8839
Epoch 188/300
112/112 [==============================] - 17s 155ms/step - loss: 0.2802 - categorical_accuracy: 0.9554 - precision_1: 0.9812 - recall_1: 0.9308 - val_loss: 0.4021 - val_categorical_accuracy: 0.8929 - val_precision

112/112 [==============================] - 8s 72ms/step - loss: 0.1153 - categorical_accuracy: 1.0000 - precision_1: 1.0000 - recall_1: 1.0000 - val_loss: 0.2642 - val_categorical_accuracy: 0.9554 - val_precision_1: 0.9550 - val_recall_1: 0.9464
Epoch 248/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1223 - categorical_accuracy: 0.9978 - precision_1: 0.9978 - recall_1: 0.9933 - val_loss: 0.5227 - val_categorical_accuracy: 0.8661 - val_precision_1: 0.8942 - val_recall_1: 0.8304
Epoch 249/300
112/112 [==============================] - 8s 70ms/step - loss: 0.2806 - categorical_accuracy: 0.9353 - precision_1: 0.9448 - recall_1: 0.9174 - val_loss: 0.3412 - val_categorical_accuracy: 0.9107 - val_precision_1: 0.9266 - val_recall_1: 0.9018
Epoch 250/300
112/112 [==============================] - 8s 73ms/step - loss: 0.2159 - categorical_accuracy: 0.9576 - precision_1: 0.9680 - recall_1: 0.9464 - val_loss: 0.3613 - val_categorical_accuracy: 0.9196 - val_precision_1: 0.94

...

Epoch 46/300
112/112 [==============================] - 18s 159ms/step - loss: 1.4852 - categorical_accuracy: 0.6384 - precision_5: 1.0000 - recall_5: 0.0045 - val_loss: 1.7462 - val_categorical_accuracy: 0.4286 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 47/300
112/112 [==============================] - 18s 159ms/step - loss: 1.4632 - categorical_accuracy: 0.6451 - precision_5: 1.0000 - recall_5: 0.0067 - val_loss: 1.6205 - val_categorical_accuracy: 0.5268 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 48/300
112/112 [==============================] - 18s 159ms/step - loss: 1.4493 - categorical_accuracy: 0.6496 - precision_5: 1.0000 - recall_5: 0.0089 - val_loss: 1.6131 - val_categorical_accuracy: 0.5268 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 49/300
112/112 [==============================] - 18s 158ms/step - loss: 1.4257 - categorical_accuracy: 0.6696 - precision_5: 1.0000 - recall_5: 0.0112 - val_loss: 1.6032 - val_categorica

112/112 [==============================] - 16s 145ms/step - loss: 0.5478 - categorical_accuracy: 0.9353 - precision_5: 0.9862 - recall_5: 0.7969 - val_loss: 0.8701 - val_categorical_accuracy: 0.7768 - val_precision_5: 0.9375 - val_recall_5: 0.6696
Epoch 109/300
112/112 [==============================] - 16s 146ms/step - loss: 0.5352 - categorical_accuracy: 0.9554 - precision_5: 0.9835 - recall_5: 0.7969 - val_loss: 0.8044 - val_categorical_accuracy: 0.8214 - val_precision_5: 0.9625 - val_recall_5: 0.6875
Epoch 110/300
112/112 [==============================] - 17s 149ms/step - loss: 0.6118 - categorical_accuracy: 0.8839 - precision_5: 0.9738 - recall_5: 0.7478 - val_loss: 0.8135 - val_categorical_accuracy: 0.7946 - val_precision_5: 0.9500 - val_recall_5: 0.6786
Epoch 111/300
112/112 [==============================] - 15s 138ms/step - loss: 0.5295 - categorical_accuracy: 0.9442 - precision_5: 0.9809 - recall_5: 0.8013 - val_loss: 0.8028 - val_categorical_accuracy: 0.8393 - val_precision

112/112 [==============================] - 15s 134ms/step - loss: 0.2172 - categorical_accuracy: 0.9799 - precision_5: 0.9909 - recall_5: 0.9688 - val_loss: 0.5377 - val_categorical_accuracy: 0.8571 - val_precision_5: 0.9100 - val_recall_5: 0.8125
Epoch 171/300
112/112 [==============================] - 15s 137ms/step - loss: 0.1993 - categorical_accuracy: 0.9866 - precision_5: 0.9932 - recall_5: 0.9799 - val_loss: 0.4742 - val_categorical_accuracy: 0.8661 - val_precision_5: 0.9495 - val_recall_5: 0.8393
Epoch 172/300
112/112 [==============================] - 17s 148ms/step - loss: 0.1838 - categorical_accuracy: 0.9888 - precision_5: 0.9955 - recall_5: 0.9844 - val_loss: 0.4670 - val_categorical_accuracy: 0.8839 - val_precision_5: 0.9406 - val_recall_5: 0.8482
Epoch 173/300
112/112 [==============================] - 15s 137ms/step - loss: 0.1817 - categorical_accuracy: 0.9866 - precision_5: 0.9977 - recall_5: 0.9821 - val_loss: 0.4680 - val_categorical_accuracy: 0.8661 - val_precision

112/112 [==============================] - 15s 134ms/step - loss: 0.0869 - categorical_accuracy: 0.9978 - precision_5: 0.9978 - recall_5: 0.9911 - val_loss: 0.3947 - val_categorical_accuracy: 0.9107 - val_precision_5: 0.9327 - val_recall_5: 0.8661
Epoch 233/300
112/112 [==============================] - 16s 139ms/step - loss: 0.0864 - categorical_accuracy: 0.9978 - precision_5: 0.9978 - recall_5: 0.9911 - val_loss: 0.3866 - val_categorical_accuracy: 0.9018 - val_precision_5: 0.9143 - val_recall_5: 0.8571
Epoch 234/300
112/112 [==============================] - 15s 130ms/step - loss: 0.4187 - categorical_accuracy: 0.8862 - precision_5: 0.9052 - recall_5: 0.8527 - val_loss: 0.6519 - val_categorical_accuracy: 0.7946 - val_precision_5: 0.8526 - val_recall_5: 0.7232
Epoch 235/300
112/112 [==============================] - 13s 117ms/step - loss: 0.3090 - categorical_accuracy: 0.9308 - precision_5: 0.9420 - recall_5: 0.9062 - val_loss: 0.3853 - val_categorical_accuracy: 0.9018 - val_precision

112/112 [==============================] - 12s 106ms/step - loss: 0.0502 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.2754 - val_categorical_accuracy: 0.9464 - val_precision_5: 0.9459 - val_recall_5: 0.9375
Epoch 295/300
112/112 [==============================] - 12s 110ms/step - loss: 0.0496 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.2568 - val_categorical_accuracy: 0.9464 - val_precision_5: 0.9636 - val_recall_5: 0.9464
Epoch 296/300
112/112 [==============================] - 11s 95ms/step - loss: 0.0495 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.2671 - val_categorical_accuracy: 0.9375 - val_precision_5: 0.9541 - val_recall_5: 0.9286
Epoch 297/300
112/112 [==============================] - 11s 100ms/step - loss: 0.0493 - categorical_accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - val_loss: 0.2468 - val_categorical_accuracy: 0.9464 - val_precision_

In [9]:
print("Loss:", loss)
print("Accuracies", accuracies)
print("Precisions", precisions)
print("Recalls", recalls)

Loss: [0.29564645886421204, 0.1803608387708664, 0.17900295555591583, 0.23405848443508148, 0.20039500296115875]
Accuracies [0.9357143044471741, 0.9714285731315613, 0.9857142567634583, 0.949999988079071, 0.9428571462631226]
Precisions [0.9629629850387573, 0.9784172773361206, 0.9852941036224365, 0.9568345546722412, 0.9635036587715149]
Recalls [0.9285714030265808, 0.9714285731315613, 0.9571428298950195, 0.949999988079071, 0.9428571462631226]


In [10]:
print("Mean Loss:", sum(loss)/len(loss))
print("Mean Accuracies", sum(accuracies)/len(accuracies))
print("Mean Precisions", sum(precisions)/len(precisions))
print("Mean Recalls", sum(recalls)/len(recalls))

Mean Loss: 0.2178927481174469
Mean Accuracies 0.9571428537368775
Mean Precisions 0.9694025158882141
Mean Recalls 0.949999988079071


In [7]:
model = Sequential() # changed activation to tanh
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.load_weights('lstmfinal.h5')

In [1]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)

In [ ]:
from scipy import stats
colors = [(245,117,16), (117,245,16), (16,117,245),(100,100,100)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [8]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'mp_holistic' is not defined